In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import Counter
from itertools import combinations_with_replacement as combos
from itertools import permutations as perms
from tensorflow.keras import layers, Model
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import coverage_error, f1_score, label_ranking_average_precision_score, average_precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from category_encoders import OneHotEncoder
from tensorflow.data import Dataset


In [9]:
def is_three_pair(choice):
    choice = sorted(choice)
    return (len(choice) == 6 and choice[0] == choice[1] and
            choice[2] == choice[3] and choice[4] == choice[5])


def is_straight(choice):
    return sorted(choice) == list(range(1, 7))


def score_all():
    return [1.] * 6


def choose_dice(roll):
    """Choose dice according to scoring rules. Boop Beep."""
    counts = Counter(roll)
    if is_three_pair(roll) and (sum(scoring_rules[die - 1][count - 1] for die, count in counts.items()) < 1500):
        choice = score_all()
    elif is_straight(roll):
        choice = score_all()
    else:
        picks = set()
        for die, count in counts.items():
            if scoring_rules[die - 1][count - 1] > 0:
                picks.add(die)
        choice = [0.] * 6
        for i, x in enumerate(roll):
            if x in picks:
                choice[i] = 1.
    return np.array(choice)


scoring_rules = [[100, 200, 1000, 2000, 4000, 5000],
                 [0, 0, 200, 400, 800, 5000],
                 [0, 0, 300, 600, 1200, 5000],
                 [0, 0, 400, 800, 1600, 5000],
                 [50, 100, 500, 1000, 2000, 5000],
                 [0, 0, 600, 1200, 2400, 5000]
                 ]


In [10]:
def make_some_features(numbers, clip):
    features = set()
    combinations = (combo for combo in combos(numbers, 6))
    for i, comb in enumerate(combinations):
        if i % clip == 0:  # Keeping size reasonable
            for perm in perms(comb):
                features.add(perm)
    return features


In [11]:
def make_special_features():
    special_features = set()
    for _ in range(500):
        half = [np.random.randint(1, 6) for _ in range(3)]
        half += half
        for perm in perms(half):
            special_features.add(perm)

    for perm in perms([1, 2, 3, 4, 5, 6]):
        special_features.add(perm)
    
    return special_features


In [12]:
features = make_some_features(list(range(1, 7)), 2)
special_features = make_special_features()

all_features = [np.array(feature) for feature in features]
print(len(all_features))
all_features += [np.array(feature) for feature in special_features]
print(len(all_features))
all_labels = [choose_dice(feature) for feature in all_features]
all_features = np.array(all_features)
all_labels = np.array(all_labels)
len(all_features), len(all_labels)

23114
25039


(25039, 25039)

In [13]:
def create_dataset(features, labels):
    data = {str(i): features[:,i] for i in range(6)}
    dataset = pd.DataFrame(data)
    label = {'{}_l'.format(i): labels[:,i] for i in range(6)}
    label_df = pd.DataFrame(label)
    df = pd.concat([dataset, label_df], axis=1, sort=False)
    return df

In [14]:
df = create_dataset(all_features, all_labels)

In [15]:
df.sample(10)

,0,1,2,3,4,5,0_l,1_l,2_l,3_l,4_l,5_l
14019,6,3,2,1,6,6,1.0,0.0,0.0,1.0,1.0,1.0
1992,3,3,2,3,4,4,1.0,1.0,0.0,1.0,0.0,0.0
11946,6,3,5,1,1,1,0.0,0.0,1.0,1.0,1.0,1.0
15116,3,4,1,6,6,2,0.0,0.0,1.0,0.0,0.0,0.0
19386,5,2,4,6,1,5,1.0,0.0,0.0,0.0,1.0,1.0
6411,3,6,2,3,6,1,0.0,0.0,0.0,0.0,0.0,1.0
24991,2,6,1,4,5,3,1.0,1.0,1.0,1.0,1.0,1.0
24933,2,2,4,4,1,1,1.0,1.0,1.0,1.0,1.0,1.0
5409,2,4,5,3,2,4,0.0,0.0,1.0,0.0,0.0,0.0
18410,2,2,5,5,2,3,1.0,1.0,1.0,1.0,1.0,0.0


In [16]:
X = df[['0', '1', '2', '3', '4', '5']]
y = df[['0_l', '1_l', '2_l', '3_l', '4_l', '5_l']]

In [17]:
def train_val_test_split(X, 
                         y,
                         train_size=.8,
                         val_size=.1,
                         test_size=.1,
                         random_state=42,
                         shuffle=True):
    
    assert train_size + val_size + test_size == 1
        

    X_trainval, X_test, y_trainval, y_test = train_test_split(X,
                                                              y,
                                                              test_size=test_size,
                                                              random_state=random_state,
                                                              shuffle=shuffle,
                                                              stratify=y
                                                             )
    X_train, X_val, y_train, y_val = train_test_split(X_trainval,
                                                      y_trainval,
                                                      test_size=val_size / (train_size + val_size),
                                                      random_state=random_state,
                                                      shuffle=shuffle,
                                                      stratify=y_trainval
                                                     )
    return X_train, X_val, X_test, y_train, y_val, y_test


In [18]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y)

In [19]:
X_train.shape, y_train.shape

((20031, 6), (20031, 6))

In [20]:
X_val.shape, y_val.shape

((2504, 6), (2504, 6))

In [21]:
X_test.shape, y_test.shape

((2504, 6), (2504, 6))

In [22]:
# model = RandomForestClassifier(n_estimators=1000,
#                                max_depth=20,
#                                min_samples_split=3,
#                                n_jobs=-1,
#                                )

In [23]:
# model.fit(X_train, y_train)

In [24]:
# model.score(X_val, y_val)

In [25]:
def test_model_pred(model):
    start = np.random.randint(0, 1392)
    print(start)
    stop = start + 25
    for test in all_features[start:stop]:
        print(test)
        true = choose_dice(test)
        print(true)
        pred = model.predict([list(test)])[0]
        print(pred)
        result = 'Nailed it' if list(true) == list(pred) else 'Nuts'
        print(result)
        print

In [26]:
# num = np.random.randint(0, 2302)
# print(num)
# test_model_pred(model, num, num + 20)

In [27]:
extra = ExtraTreesClassifier(bootstrap=True,
                             max_depth=25,
                             n_jobs=-1,
                             min_samples_split=3,
                             n_estimators=400)
extra.fit(X_train, y_train)
extra.score(X_val, y_val)

0.7480031948881789

In [28]:
test_model_pred(extra)

979
[2 5 1 3 1 4]
[0. 1. 1. 0. 1. 0.]
[0. 1. 1. 0. 1. 0.]
Nailed it

[1 4 1 6 3 6]
[1. 0. 1. 0. 0. 0.]
[1. 0. 1. 0. 0. 0.]
Nailed it

[4 3 4 2 4 6]
[1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0.]
Nailed it

[5 4 3 6 1 4]
[1. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 1. 0.]
Nailed it

[5 2 3 2 5 2]
[1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
Nuts

[1 2 6 6 3 2]
[1. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0.]
Nailed it

[1 4 1 4 5 2]
[1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0.]
Nailed it

[6 6 4 1 2 3]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
Nailed it

[3 2 2 5 4 4]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
Nailed it

[6 1 3 2 6 4]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
Nailed it

[2 3 6 6 5 5]
[0. 0. 0. 0. 1. 1.]
[0. 0. 0. 0. 1. 1.]
Nailed it

[3 3 1 6 6 5]
[0. 0. 1. 0. 0. 1.]
[0. 0. 1. 0. 0. 1.]
Nailed it

[1 3 3 5 2 5]
[1. 0. 0. 1. 0. 1.]
[1. 0. 0. 1. 0. 1.]
Nailed it

[4 5 4 5 1 2]
[0. 1. 0. 1. 1. 0.]
[0. 1. 0. 1. 1. 0.]
Nailed it

[1 1 6 5 5 3]
[1. 1. 0. 1. 1. 0.]
[1. 1. 0. 1. 1. 0.]
Nailed it

[6 4 5 3 5 4]
[0. 0. 1. 0.

In [29]:
# params = {'n_estimators': [1000, 1500, 2000, 3000]}

# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='average_precision',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [30]:
# params = {'max_depth': [15, 20, 25, 30]}
# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='f1_samples',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [31]:
# params = {'min_samples_split': [5, 6, 7, 8]}
# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='f1_samples',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [32]:
# params = {'oob_score': [True, False]}
# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='f1_samples',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [33]:
params = {'min_samples_split': [4, 5, 6],
          'max_depth': [27, 30, 33],
          'oob_score': [True],
          'n_estimators': [2000]}
grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 47.3min finished


({'max_depth': 33,
  'min_samples_split': 6,
  'n_estimators': 2000,
  'oob_score': True},
 0.9608719752519872)

In [34]:
# params = {'min_samples_split': [3, 4, 5],
#           'max_depth': [26, 27, 28],
#           'oob_score': [True],
#           'n_estimators': [2000]}
# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='f1_samples',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [35]:
params = {'n_estimators': [1250, 1500, 1750, 2000, 2250, 2500]}
grid = GridSearchCV(grid.best_estimator_,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 28.5min finished


({'n_estimators': 2250}, 0.9608590931565371)

In [36]:
best = grid.best_estimator_

In [37]:
best.score(X_val, y_val)

0.7651757188498403

In [52]:
test_model_pred(best)

1105
[5 5 2 2 4 3]
[1. 1. 0. 0. 0. 0.]
[1. 1. 0. 0. 0. 0.]
Nailed it

[3 1 1 4 2 2]
[0. 1. 1. 0. 0. 0.]
[0. 1. 1. 0. 0. 0.]
Nailed it

[4 2 3 6 6 5]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
Nailed it

[1 2 4 1 3 3]
[1. 0. 0. 1. 0. 0.]
[1. 0. 0. 1. 0. 0.]
Nailed it

[6 3 5 5 1 1]
[0. 0. 1. 1. 1. 1.]
[0. 0. 1. 1. 1. 1.]
Nailed it

[4 5 2 4 2 5]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
Nailed it

[4 3 4 3 4 2]
[1. 0. 1. 0. 1. 0.]
[1. 1. 1. 1. 1. 0.]
Nuts

[6 5 4 3 2 4]
[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
Nailed it

[5 6 6 3 6 6]
[1. 1. 1. 0. 1. 1.]
[1. 1. 1. 0. 1. 1.]
Nailed it

[6 2 1 4 6 6]
[1. 0. 1. 0. 1. 1.]
[1. 0. 1. 0. 1. 1.]
Nailed it

[3 1 3 1 6 6]
[1. 1. 1. 1. 1. 1.]
[0. 1. 0. 1. 0. 0.]
Nuts

[1 3 2 5 3 4]
[1. 0. 0. 1. 0. 0.]
[1. 0. 0. 1. 0. 0.]
Nailed it

[5 4 1 2 3 4]
[1. 0. 1. 0. 0. 0.]
[1. 0. 1. 0. 0. 0.]
Nailed it

[1 3 4 1 6 1]
[1. 0. 0. 1. 0. 1.]
[1. 0. 0. 1. 0. 1.]
Nailed it

[6 2 1 6 4 2]
[0. 0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
Nailed it

[4 6 3 3 3 3]
[0. 0. 1. 1. 1. 

In [39]:
y_pred = [best.predict([test])[0] for test in X_test.values]

In [40]:
y_test.iloc[0]

0_l    0.0
1_l    1.0
2_l    1.0
3_l    1.0
4_l    1.0
5_l    0.0
Name: 3165, dtype: float64

In [41]:
y_pred[:5]

[array([0., 1., 1., 1., 1., 0.]),
 array([0., 1., 1., 0., 1., 1.]),
 array([0., 0., 0., 0., 1., 0.]),
 array([1., 0., 1., 1., 1., 0.]),
 array([1., 0., 0., 0., 1., 1.])]

In [42]:
y_pred = np.array(y_pred)

In [43]:
coverage_error(y_pred, y_test)

3.4253194888178915

In [44]:
f1_score(y_test, y_pred, average= 'samples')

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


0.8899429311370205

In [45]:
y_pred.shape, y_test.shape

((2504, 6), (2504, 6))

In [46]:
y_test.sum()

0_l    1358.0
1_l    1358.0
2_l    1358.0
3_l    1358.0
4_l    1357.0
5_l    1359.0
dtype: float64

In [47]:
y_test.sum().sum() / len(y_test)

3.253993610223642

In [48]:
label_ranking_average_precision_score(y_test, y_pred)

0.9639687610933619

In [49]:
average_precision_score(y_test, y_pred)

0.8914703279666899